In [1]:
import argilla as rg

from datasets import load_dataset, Dataset
from setfit import SetFitModel, SetFitTrainer, get_templated_dataset, sample_dataset
import spacy_fastlang
import spacy



2024-08-13 12:17:36.598408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import os

client = rg.Argilla(
    api_url=os.environ["ARGILLA_API_URL"],
    api_key=os.environ["ARGILLA_API_KEY"]
)

In [3]:
dataset = client.datasets("homefitness_dataset_v1")

In [140]:
records = dataset.records().to_list(flatten=True)

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/argilla/_models/_record/_response.py:43: UserWarning: `user_id` not provided, so it will be set to `None`. Which is not an issue, unless you're planning to log the response in Argilla, as it will be automatically set to the active `user_id`.
  warnings.warn(


In [5]:
def detect_language(text):
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("language_detector")
    doc = nlp(text)
    if(doc._.language== "en"):
        return True
    else:
        return False


In [141]:
train_records = []
del_records = []
for record in records:
    text = record["title"] + record["description"]
    #if(not detect_language(text)):
    #    del_records.append(record)
    #    continue
    if( record['fit_label.responses'] is not None and record['fit_label.responses'][0] == 'Yes'):
        label = 1
    else:
        label = 0
    train_records.append({"text": text, "label": label})

In [8]:
for rec in del_records:
    records_to_delete = list(dataset.records(query=rec["video_id"]))
    if(len(records_to_delete) > 0):
        dataset.records.delete(records_to_delete)

In [142]:
train_records = [
    {
        "text": " ".join(r["text"].split()),
        "label": r["label"],
    }
    for r in train_records
]

In [119]:
train_dataset = Dataset.from_list(train_records)
#train_dataset = sample_dataset(train_dataset, label_column="label", num_samples=8)


In [19]:
for row in train_dataset:
    print(row)

{'text': 'Kira BMR sgt penting utk lossweight journey / apa beza BMI dan BMR? ', 'label': 'No'}
{'text': 'AT HOME WorkoutsFull Bodyweight Circuit!', 'label': 'Yes'}
{'text': 'CANNIBAL – Ke$ha  || cardio dance fitnessCannibal – Ke$ha  || GetFit with J \n\n\n** Video is for instructional use only. I do not own the rights to the song.** \n\nChoreo: TeamBernFitness \n\n📍 Local and want to check out a class....', 'label': 'Yes'}
{'text': 'LEG WORKOUT | All Standing LEG/BUTT/THIGH Home Workout - No EquipmentAll Standing LEG/BUTT/THIGH Home Workout (No Equipment Needed)\n\nSometimes you just need a quick, down and dirty, leg shaking leg workout.  AND one that doesn’t require a butt-ton of workout...', 'label': 'Yes'}
{'text': "simple Home workouts to loose weight very fast.Join my sister's home workouts and healthy  eating to loose weight faster.\n\nmusic in video provided by Senait G Mehari \nplease check her out on YouTube,Spotify,sound cloud,Instagram and more......", 'label': 'Yes'}
{'tex

In [41]:
labels = ["positive", "negative"]

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [107]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [121]:
train_dataset =train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4996 [00:00<?, ? examples/s]

Map:   0%|          | 0/1249 [00:00<?, ? examples/s]

In [145]:
Dataset.to_csv(train_dataset, "dataset.csv")

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

1451720

In [144]:
train_dataset = Dataset.from_list(train_records)


In [120]:
stratify_column_name = "label"

train_dataset= train_dataset.class_encode_column(
    stratify_column_name
).train_test_split(test_size=0.2,stratify_by_column=stratify_column_name)

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Stringifying the column:   0%|          | 0/6245 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/6245 [00:00<?, ? examples/s]

In [130]:
c_yes =0 
c_no = 0
for row in train_dataset['train']:
    if(row["label"] == 1):
        c_yes += 1
    else:    
        c_no += 1

In [123]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [64]:
import evaluate

accuracy = evaluate.load("accuracy")

In [111]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [112]:
id2label = {0: "No", 1: "Yes"}
label2id = {"No": 0, "Yes": 1}

In [113]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [132]:
training_args = TrainingArguments(
    output_dir="y_fit_classifier_v1",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset["train"],
    eval_dataset=train_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/471 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [115]:
trainer.save_model("yt_fit_classifier_v1")